In [93]:
import ujson as json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
import contractions



In [94]:
df = pd.read_json("ext_act_lang_filtered.ndjson", lines=True)

Pre-processing order
1. tokenize 
2. downcase
3. punct
4. stopwords
5. lemmatize
6. realtive pruning

In [1]:
import string
punct = string.punctuation
punct = punct.replace("#", "‘") #removing hashtag and replacing with a stray type of apostrophe that broke through before


In [2]:
punct

'!"‘$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [114]:
# global vars, but not too horrible I figure
stopwords = nltk.corpus.stopwords.words("english")
import string
punct = string.punctuation
punct = punct.replace("#", "‘") #removing hashtag and replacing with a stray type of apostrophe that broke through before
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

# defining a pre-proc function that will tokenize text
def pre_proc(x):
    # splitting contractions before splitting up the terms => helps with pruning
    # trying to expand the contractions
    try:    
        x = contractions.fix(x)
    
    # if that fails, trying to use unidecode to correct
    except IndexError:
        # trying to unidecode
        try:
            x = contractions.fix(unidecode(x))
        # printing excpetion, and input leading to that
        except Exception as eror:
            print("Exception is:", error)
            print("Exception on:")
            print(x)        
        
    # splitting the terms up
    x = x.split()
    
    # removing the empty items
    x = [word for word in x if len(word)>0]

    # lowercasing the terms
    x = [word.lower() for word in x]
    
    # removing emoji & conveniently, some punctuation
    x = [word.encode("ascii", "ignore").decode("ascii") for word in x]

    # removing punctuation (with the exception of #)
    x = [word.translate(str.maketrans("","",punct)) for word in x]    
       
    # keeping numbers because they *might* prove important -> avoid being overly aggressive

    # removing the empty items
    x = [word for word in x if len(word)>0]
    
    # removing stopwords
    x = [word for word in x if word not in stopwords]
    
    # lemmatization
    x = [wordnet_lemmatizer.lemmatize(word) for word in x]

    # rejoining the list back into a string   
    x = " ".join(x)  

    return x

In [115]:
def file_name(file):
    # splitting on "."
    file = file.split(".")
    # adding the name to show it's been filtered
    file[0] = file[0] + "_pre_proc"
    # ading the file type
    file = file[0] + "." + file[1]
    # returning the file
    return file

In [116]:
# file_list
file_list = ["min_act_lang_filtered.ndjson","mod_act_lang_filtered.ndjson",
             "ver_act_lang_filtered.ndjson","ext_act_lang_filtered.ndjson"]

In [117]:
%%time
# cycling through the file_list
for file in file_list:
    # printing the file name
    print(file)
    
    # reading in the file
    df = pd.read_json(file, lines=True)
    
    # applying the pre-proc function
    df["bo"] = df["bo"].apply(pre_proc)
    
    # changing the lang file
    f_name = file_name(file)
    
    df.to_json(f_name, orient="records",lines=True)

min_act_lang_filtered.ndjson
mod_act_lang_filtered.ndjson
ver_act_lang_filtered.ndjson
ext_act_lang_filtered.ndjson
CPU times: total: 2h 2min 17s
Wall time: 2h 3min 21s
